In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load .env and DB
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)

# Load merged data
df = pd.read_sql("SELECT * FROM merged_features", engine, parse_dates=["date"])

# Sort before shifting
df = df.sort_values(["ticker", "date"])

# Calculate 5-day forward return
df["future_close"] = df.groupby("ticker")["close"].shift(-5)
df["target_return_5d"] = (df["future_close"] - df["close"]) / df["close"]

# Drop helper column
df.drop(columns=["future_close"], inplace=True)

# Save updated features
df.to_sql("merged_features", engine, if_exists="replace", index=False)
print(f"✅ target_return_5d added to merged_features table.")

✅ target_return_5d added to merged_features table.
